In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

**SCRAPING**

In [2]:
nifty50=[
    "ADANIGREEN", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BEL", "BHARTIARTL",
    "BPCL", "CIPLA", "COALINDIA", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO",
    "HINDALCO", "HINDUNILVR", "ICICIBANK", "INDUSINDBK", "INFY",
    "IOC", "ITC", "JSWSTEEL", "KOTAKBANK", "LT", "M&M",
    "MARUTI", "NESTLEIND", "NTPC", "ONGC", "POWERGRID",
    "RELIANCE", "SBIN", "SBILIFE", "SHRIRAMFIN", "SUNPHARMA",
    "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TCS", "TECHM",
    "TITAN", "TRENT", "ULTRACEMCO", "WIPRO"
]


In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


def ticker_info(ticker):

    url = f'https://www.screener.in/company/{ticker}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    top_section = soup.find("div", {"id": "top"})
    pnl_section = soup.find("section", {"id": "profit-loss"})

    ratios = top_section.find("ul", {"id": "top-ratios"}).find_all("li")[0:4]
    values = [ratio.find("span", {"class": "number"}).text for ratio in ratios]
    col_names = ["MarketCap","Volume", "LTP", "High", "Low", "PE"]

    ratios_dict = dict(zip(col_names, values))

    input_string = top_section.find("span", {"class": "font-size-12"}).get_text()
    match = re.search(r"-?\s*\d+\.\d+", input_string)
    if match:
        ratios_dict["PercentChange"] = float(match.group().strip())

    try:
        eps_value = pnl_section.find_all("tr")[11].find_all("td")[12].text
        ratios_dict["EPS"] = eps_value
    except (IndexError, AttributeError):
        ratios_dict["EPS"] = "N/A"

    return ratios_dict


In [5]:
df_ = []
for stock in nifty50:
  details = ticker_info(stock)
  df_.append({"ticker": stock, **details,})


In [6]:
df_=pd.DataFrame(df_)
df_

,ticker,MarketCap,Volume,LTP,High,PercentChange,EPS
0,ADANIGREEN,"1,89,830","1,198","2,174","14,647",-1.61,N/A
1,ADANIPORTS,"2,72,167","1,260","1,621",109,1.28,8.05
2,APOLLOHOSP,"1,04,380","7,259","7,545",93.0,0.45,70.28
3,ASIANPAINT,"2,30,941","2,408","3,423",50.3,0.76,55.48
4,AXISBANK,"3,55,330","1,148","1,340",13.6,0.22,80.55
5,BAJAJ-AUTO,"2,51,929","9,021","12,774",30.8,0.65,267.88
6,BAJFINANCE,"4,44,613","7,183","7,830",32.7,0.80,204.27
7,BAJAJFINSV,"2,68,191","1,680","2,030",173,0.02,7.33
8,BEL,"2,30,733",316,340,50.8,0.86,5.50
9,BHARTIARTL,"10,07,452","1,682","1,779",133,4.42,8.82


USING YAHOO FINANCE

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def calculate_returns(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker + ".NS", start=start_date, end=end_date, progress=False)
        if stock_data.empty:
            return None
        adj_close = stock_data['Adj Close']
        if not adj_close.empty:
            start_price = adj_close.iloc[0].item()
            end_price = adj_close.iloc[-1].item()
            return_pct = ((end_price - start_price) / start_price) * 100
            return round(return_pct, 2)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
    return None

# Date ranges for 6 months, 1 year, and 5 years
end_date = datetime.now()
six_months_ago = end_date - timedelta(days=180)
one_year_ago = end_date - timedelta(days=365)
five_years_ago = end_date - timedelta(days=5 * 365)


stock_data_list = []
for ticker in nifty50:
    six_month_return = calculate_returns(ticker, six_months_ago.date(), end_date.date())
    one_year_return = calculate_returns(ticker, one_year_ago.date(), end_date.date())
    five_year_return = calculate_returns(ticker, five_years_ago.date(), end_date.date())

    try:
        details = ticker_info(ticker)
    except Exception as e:
        print(f"Error getting details for {ticker}: {e}")
        details = {}
        #merging
    stock_data_list.append({
        "Ticker": ticker,
        **details,
        "6 Month Return": six_month_return,
        "1 Year Return": one_year_return,
        "5 Year Return": five_year_return
    })


df = pd.DataFrame(stock_data_list)


# print(df)


In [8]:
df

,Ticker,MarketCap,Volume,LTP,High,PercentChange,EPS,6 Month Return,1 Year Return,5 Year Return
0,ADANIGREEN,"1,89,830","1,198","2,174","14,647",-1.61,N/A,-33.94,-21.40,784.10
1,ADANIPORTS,"2,72,167","1,260","1,621",109,1.28,8.05,-12.81,15.63,251.77
2,APOLLOHOSP,"1,04,380","7,259","7,545",93.0,0.45,70.28,16.58,32.34,439.99
3,ASIANPAINT,"2,30,941","2,408","3,423",50.3,0.76,55.48,-17.36,-26.92,43.49
4,AXISBANK,"3,55,330","1,148","1,340",13.6,0.22,80.55,-3.60,3.39,54.65
5,BAJAJ-AUTO,"2,51,929","9,021","12,774",30.8,0.65,267.88,-9.04,40.66,228.63
6,BAJFINANCE,"4,44,613","7,183","7,830",32.7,0.80,204.27,-1.58,-4.86,76.62
7,BAJAJFINSV,"2,68,191","1,680","2,030",173,0.02,7.33,5.18,-2.54,79.84
8,BEL,"2,30,733",316,340,50.8,0.86,5.50,-0.55,84.62,949.02
9,BHARTIARTL,"10,07,452","1,682","1,779",133,4.42,8.82,18.40,70.78,296.57
